## Tema 10: Numba. Cython. Ufuncs en Numba (soluciones)

## A - Numba

__Suma de Riemann__

In [ ]:
import numpy as np
from numba import jit

def riemann_sum(n):
    dx = (np.pi/2 - 0) / n # Ancho de los intervalos
    riemann_sum = 0
    for i in range(n):
        x = (i + 0.5) * dx  # Midpoint Riemann sum
        riemann_sum += np.sin(x) * dx
    return riemann_sum
    
@jit
def riemann_sum_numba(n):
    dx = (np.pi/2 - 0) / n # Ancho de los intervalos
    riemann_sum = 0
    for i in range(n):
        x = (i + 0.5) * dx  # Midpoint Riemann sum
        riemann_sum += np.sin(x) * dx
    return riemann_sum

def riemann_sum_noloops(n):
    dx = (np.pi/2 - 0) / n # Ancho de los intervalos
    points = np.arange(dx/2, np.pi/2, dx)
    riemann_sum = np.sum(dx*np.sin(points))
    return riemann_sum

@jit
def riemann_sum_noloops_numba(n):
    dx = (np.pi/2 - 0) / n # Ancho de los intervalos
    points = np.arange(dx/2, np.pi/2, dx)
    riemann_sum = np.sum(dx*np.sin(points))
    return riemann_sum


In [ ]:
%timeit riemann_sum(2000)
%timeit riemann_sum_noloops(2000)
%timeit riemann_sum_numba(2000)
%timeit riemann_sum_noloops_numba(2000)

## B - Cython

__Suma de Riemann__

In [ ]:
# riemann_sum_cython.pyx

cimport numpy
import numpy as np

def riemann_sum_cython(int n):

  cdef:
    double dx
    int i
    double x
    double riemann_sum = 0


  dx = (np.pi/2 -0) / n
  
  for i in range(n):
    x = (i + 0.5) * dx  # Midpoint Riemann sum
    riemann_sum += np.sin(x) * dx
  return riemann_sum

In [ ]:
# riemann_sum_cython_setup.py

from setuptools import setup
from Cython.Build import cythonize
import numpy

setup(
    ext_modules = cythonize("riemann_sum_cython.pyx"),
    include_dirs=[numpy.get_include()]
)

In [ ]:
# riemann_sum_noloops_cython.pyx

cimport numpy
import numpy as np

def riemann_sum_noloops_cython(int n):

  cdef:
    double dx = (np.pi/2 -0) / n
    points = np.arange(dx/2, np.pi/2, dx)
  
  riemann_sum = np.sum(dx*np.sin(points))
  return riemann_sum

In [ ]:
# riemann_sum_noloops_cython_setup.py

from setuptools import setup
from Cython.Build import cythonize
import numpy

setup(
    ext_modules = cythonize("riemann_sum_noloops_cython.pyx"),
    include_dirs=[numpy.get_include()]
)

## C - Ufuncs en Numba

__Ejercicio 3: vectorizar función en la GPU__

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from numba import vectorize

@vectorize(['int16(int16, int16)'], target='cuda')
def zero_suppress(waveform_value, threshold):
    if waveform_value < threshold:
        result = 0
    else:
        result = waveform_value
    return result

n = 100000
noise = np.random.normal(size=n) * 3
pulses = np.maximum(np.sin(np.arange(n) / (n / 23)) - 0.3, 0.0)
noisy_signal = ((pulses * 300) + noise).astype(np.int16)

clean_signal = zero_suppress(noisy_signal, 15)

fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1)
ax1.plot(noisy_signal)
ax1.set_ylabel("Señal ruidosa")
ax2.plot(clean_signal)
ax2.set_ylabel("Señal limpia")
fig.tight_layout()
fig.show()

__Ejercicio 4: optimizar la gestión de la memoria__

In [ ]:
import math
from numba import vectorize, cuda
from matplotlib import pyplot as plt
import numpy as np

@vectorize(['float32(float32, float32, float32)'], target='cuda')
def make_pulses(i, period, amplitude):
    return max(math.sin(i / period) - 0.3, 0.0) * amplitude

@vectorize(['float32(float32, float32)'], target='cuda')
def add_ufunc(x, y):
    return x + y

n = 100000
period = n / 23
h_noise = (np.random.normal(size=n) * 3).astype(np.float32)
t = np.arange(n, dtype=np.float32)


d_pulses = cuda.device_array(shape=(n,), dtype=np.float32)
d_noise=cuda.to_device(h_noise)
d_t=cuda.to_device(t)

make_pulses(d_t, period, 100.0, out=d_pulses)
waveform = add_ufunc(d_pulses, d_noise)


fig, ax = plt.subplots(nrows=1, ncols=1)
ax.plot(waveform)
fig.show())